In [17]:
import pandas as pd
import re
import requests
from bs4 import BeautifulSoup

def most_active_stocks(url):
     
    #Para descargar el contenido de la pagina
    import requests
    r=requests.get(url)

    #Creamos el árbol de objetos Python que representan al documento HTML (la sopa)
    soup = BeautifulSoup(r.text, 'lxml')
    
    #Tenemos que "engañar" a la pagina, para que crea que somos un navegador, por ello, incluimos headers, sino, obtendriamos el siguiente error:
    #<title>403 You are banned from this site.  Please contact via a different client configuration if you believe that this is a mistake.</title>

    header={'User-Agent':'Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/41.0.2227.0 Safari/537.36'}
    page=requests.get(url,headers=header)

    # Creamos las listas y el diccionario para recoger el data set
    list_name = []
    list_last = []
    list_high = []
    list_vol = []
    list_low = []
    list_time = []
    list_var = []
    list_por_var = []
    dict_data = {}
    
    # Comprobamos que la petición nos devuelve un Status Code = 200
    status_code = page.status_code
    if status_code == 200:
        
        soup=BeautifulSoup(page.content,'html.parser')
        result = soup.find('td',attrs={'class':'left bold plusIconTd elp'})
    
        # Crearemos un bucle que vaya recogiendo los datos y los vaya metiendo a sus corresponeintes listas
        for link in soup.find_all('td',attrs={'class':'left bold plusIconTd elp'}):
            name= link.get_text()
            list_name.append(name)
            
            line = link.find('span',{'class':'alertBellGrayPlus js-plus-icon genToolTip oneliner'})
            id_valor = re.findall('\D(\d{3,7})\D', str(line))
            new_class_1 = "align_right pid-"+ id_valor[0] +"-last"
            last=soup.find('td',attrs={'class':new_class_1}).getText()
            list_last.append(last)

            new_class_2 = "pid-"+ id_valor[0] +"-time"
            time=soup.find('td',attrs={'class':new_class_2}).getText()
            list_time.append(time)

            new_class_3 = "pid-"+ id_valor[0] +"-turnover"
            vol=soup.find('td',attrs={'class':new_class_3}).getText()
            list_vol.append(vol)

            new_class_4 = "align_right pid-"+ id_valor[0] +"-high"
            high=soup.find('td',attrs={'class':new_class_4}).getText()
            list_high.append(high)

            new_class_5 = "pid-"+ id_valor[0] +"-low"
            low=soup.find('td',attrs={'class':new_class_5}).getText()
            list_low.append(low)

            if soup.find('td',attrs={'class':"bold redFont pid-"+id_valor[0]+"-pc"}) != None:
                var = soup.find('td',attrs={'class':"bold redFont pid-"+id_valor[0]+"-pc"}).getText()
                por_var = soup.find('td',attrs={'class':"bold redFont pid-"+id_valor[0]+"-pcp"}).getText()

            elif soup.find('td',attrs={'class':"bold greenFont pid-"+id_valor[0]+"-pc"}) != None:
                var = soup.find('td',attrs={'class':"bold greenFont pid-"+id_valor[0]+"-pc"}).getText()
                por_var = soup.find('td',attrs={'class':"bold greenFont pid-"+id_valor[0]+"-pcp"}).getText()

            else:
                var = soup.find('td',attrs={'class':"bold blackFont pid-"+id_valor[0]+"-pc"}).getText()
                por_var = soup.find('td',attrs={'class':"bold blackFont pid-"+id_valor[0]+"-pcp"}).getText()

            list_var.append(var)
            list_por_var.append(por_var)
            
        # Creamos las entreadas del diccionario y lo convertimos en un dataframe
        dict_data['Nombre'] = list_name
        dict_data['Último valor'] = list_last
        dict_data['Máx. valor'] = list_high
        dict_data['Min. valor'] = list_low
        dict_data['Variación'] = list_var
        dict_data['% Variación'] = list_por_var
        dict_data['Volumen'] = list_vol
        dict_data['Hora'] = list_time

        df = pd.DataFrame(dict_data, columns = ['Nombre', 'Último valor', 'Máx. valor', 'Min. valor', 'Variación', '% Variación', 'Volumen', 'Hora'])

        # Descargamos un archivo .csv 
        final_datafile = df.to_csv('WebScraping_SP_stocks.csv', sep=';')
    else:
        print("Status Code %d" % status_code)
    
    
    return
    

# Ejecutamos la función creada
most_active_stocks("https://es.investing.com/equities/most-active-stocks")


{'Nombre': ['Iberdrola', 'Santander', 'Inditex', 'Amadeus', 'Repsol', 'BBVA', 'Telefónica', 'Cellnex Telecom', 'Ferrovial', 'PharmaMar', 'Siemens Gamesa', 'Aena', 'Naturgy Energy', 'Endesa', 'Red Eléctrica', 'CaixaBank', 'MásMóvil', 'MásMóvil', 'Enagás', 'ACS', 'IAG', 'Grifols', 'Banco Sabadell', 'Bankinter', 'Merlin Properties', 'Acciona', 'Solaria', 'Mapfre', 'Acerinox', 'Indra', 'ArcelorMittal', 'Colonial', 'Cie Automotive', 'ENCE', 'Meliá Hotels', 'Bankia', 'Viscofan', 'Nyesa Valores Corp', 'Almirall', 'Sacyr', 'Logista', 'Fluidra', 'Applus', 'Audax Renovables', 'Mediaset', 'Técnicas Reunidas', 'Airbus Group', 'Rovi', 'Vidrala', 'Grifols Pref'], 'Último valor': ['10,310', '1,644', '21,680', '40,155', '5,272', '2,366', '2,969', '53,70', '19,440', '110,300', '24,220', '113,90', '16,335', '22,810', '15,605', '1,570', '22,50', '22,50', '18,960', '20,220', '1,032', '23,845', '0,261', '3,158', '5,73', '90,100', '15,740', '1,280', '6,890', '5,370', '11,606', '6,020', '15,990', '1,875', '3